In [1]:
import pandas as pd
from urllib.request import urlopen
from urllib.error import HTTPError
import json 
import numpy as np

條目總數：1331 筆

In [2]:
fname = '/home/jovyan/Projects/Workbench/TW_parachute/6.1331_TW_others_Zheng.xlsx'
df = pd.read_excel(fname)
# df

## 預處理
### 1. 剔除沒有提供 doi 的條目

沒有 doi 的條目無法查詢樣本處理和科研計畫補助的細節。

- 對其中的幾個條目試過手動以 Google 搜尋標題，結果都找不到連結。
- 剩下 1014 個條目

In [3]:
df = df.dropna(subset=['DI'])
len(df)
# df

1014

### 2. 剔除重複 doi 的條目

因為一個 doi 只能解析出一個 url 位址，因此相同的 doi 意味著某個論文連結被重複記錄到檔案裡面了。

- 剩下 964 個條目

In [4]:
_, idx = np.unique(df['DI'], return_index=True)
df = df.iloc[np.sort(idx)]
len(df)
# df

964

### 3. 搜尋 `AU_CO` 欄位找出沒有台灣作者的條目

首先先確認哪些條目的 `AU_CO` 欄位缺失，這些條目可能之後需要手動補上/搜尋。

In [5]:
df[df['AU_CO'].isna()]

,PY,JI,TI,AU,AB,C1,AU_CO,AU1_CO,AUc_CO,N_AU,...,DI,DT,ID,PU,SO,TC,VL,DB,AU_UN,AUc_UN
121,1985.0,Chem. Geol.,CHEMICAL CHARACTERISTICS OF THERMAL WATERS IN ...,CHEN C,NaN,NaN,NaN,NaN,TAIWAN,1,...,10.1016/0009-2541(85)90163-9,Article,NaN,ELSEVIER SCIENCE BV,CHEMICAL GEOLOGY,12,49,ISI,NaN,CH (CORRESPONDING AUTHOR)
152,1994.0,Desalination,SEAWATER DESALINATION TO AUGMENT THE WATERSUPP...,CHEN J;LU H;AWERBUCH L,"To alleviate water shortage on Penghu Island, ...","BECHTEL CORP,GAITHERSBURG,MD 20878.;",NaN,NaN,TAIWAN,3,...,10.1016/0011-9164(94)00096-4,Article; Proceedings Paper,NaN,ELSEVIER SCIENCE BV,DESALINATION,0,97,ISI,NaN,J (CORRESPONDING AUTHOR)
222,1997.0,Geochem. J.,Acidification of lakes and reservoirs in Taiwan,Chen C;Wang B,"One hundred forty three lakes, ponds and reser...",NaN,NaN,NaN,TAIWAN,2,...,10.2343/geochemj.31.345,Article,ACID,GEOCHEMICAL SOC JAPAN,GEOCHEMICAL JOURNAL,7,31,ISI,NaN,NATL SUN YAT SEN UNIV
242,1996.0,Geochim. Cosmochim. Acta,Variability of treering delta C13 in Taiwan fi...,Sheu D;Kou P;Chiu C;Chen M,We have analyzed the delta(13)C of tree-ring c...,NaN,NaN,NaN,TAIWAN,4,...,10.1016/0016-7037(95)00401-7,Article,ATMOSPHERIC CARBON-DIOXIDE; C-13-C-12 RATIO; I...,PERGAMON-ELSEVIER SCIENCE LTD,GEOCHIMICA ET COSMOCHIMICA ACTA,23,60,ISI,NaN,NATL SUN YAT SEN UNIV
304,1996.0,Geology,Extensional collapse of the northern Taiwan mo...,Teng L,As an active collision zone between the Luzon ...,NaN,NaN,NaN,TAIWAN,1,...,10.1130/0091-7613(1996)024<0949:ECOTNT>2.3.CO;2,Article,BACK-ARC BASIN; OKINAWA TROUGH; COLLISION; LIT...,GEOLOGICAL SOC AMERICA,GEOLOGY,205,24,ISI,NaN,NATL TAIWAN UNIV
305,1997.0,Geology,Extensional collapse of the northern Taiwan mo...,Chen C,NaN,NaN,NaN,NaN,TAIWAN,1,...,10.1130/0091-7613(1997)025<0855:ECOTNT>2.3.CO;2,Article,ARC; BASALTS; ISLAND,GEOLOGICAL SOC AMERICA,GEOLOGY,14,25,ISI,NaN,INST EARTH SCI
509,1997.0,J. Asian Earth Sci.,Kaoping Shelf Morphology and tectonic signific...,Yu H;Chiang C,Thirty bathymetric profiles across the southwe...,NaN,NaN,NaN,TAIWAN,2,...,10.1016/S0743-9547(96)00076-1,Article,TAIWAN,PERGAMON-ELSEVIER SCIENCE LTD,JOURNAL OF ASIAN EARTH SCIENCES,18,15,ISI,NaN,NATL TAIWAN UNIV
757,1997.0,J. Geodesy,Analysis of some systematic errors affecting a...,Hwang C,This paper analyzes several systematic errors ...,NaN,NaN,NaN,TAIWAN,1,...,10.1007/s001900050080,Article,SATELLITE ALTIMETRY; GRAVITY-FIELD; TOPEX/POSE...,SPRINGER VERLAG,JOURNAL OF GEODESY,28,71,ISI,NaN,NATL CHIAO TUNG UNIV
893,1996.0,J. Metamorph. Geol.,Raman spectrum of carbonaceous material A poss...,Yui T;Huang E;Xu J,Raman spectral analyses of carbonaceous materi...,NaN,NaN,NaN,TAIWAN,3,...,10.1046/j.1525-1314.1996.05792.x,Article,GRAPHITE; CALCITE; GRAPHITIZATION; SPECTROSCOP...,WILEY,JOURNAL OF METAMORPHIC GEOLOGY,140,14,ISI,NaN,INST EARTH SCI
1105,1984.0,Paleogeogr. Paleoclimatol. Paleoecol.,PLANKTIC FORAMINIFERAL BIOSTRATIGRAPHY AND DAT...,HUANG T,NaN,NaN,NaN,NaN,TAIWAN,1,...,10.1016/0031-0182(84)90028-2,Article,NaN,ELSEVIER SCIENCE BV,PALAEOGEOGRAPHY PALAEOCLIMATOLOGY PALAEOECOLOGY,15,46,ISI,NaN,T (CORRESPONDING AUTHOR)


再來對於有 `AU_CO` 紀錄的條目，搜尋 `TAIWAN` 字串。**具有此字串的條目應該不會是 C 類文章。**

In [7]:
df_good_auco = df.dropna(subset=['AU_CO'])
contains_taiwan = np.array(['TAIWAN' in row['AU_CO'] for idx, row in df_good_auco.iterrows()])
df_good_auco.loc[contains_taiwan]

,PY,JI,TI,AU,AB,C1,AU_CO,AU1_CO,AUc_CO,N_AU,...,DI,DT,ID,PU,SO,TC,VL,DB,AU_UN,AUc_UN
1,2011.0,AAPG Mem.,Two contrasting kinematic styles of active fol...,Yue L;Suppe J;Hung J,Two adjacent active thrust ramps in western Ta...,"Department of Geosciences, Princeton Universit...",USA;TAIWAN;TAIWAN;TAIWAN,USA,USA,3,...,10.1306/13251337M943431,Article,coseismic process; deformation; detachment f...,NaN,AAPG Memoir,18,NaN,scopus,PRINCETON UNIVERSITY;NATIONAL TAIWAN UNIVERSIT...,DEPARTMENT OF GEOSCIENCES
2,1977.0,Agricultural Administration,ChineseAmerican Joint Commission on Rural Reco...,Wang Y,The Chinese-American Joint Commission on Rural...,"Secretary General, Joint Commission on Rural R...",TAIWAN,TAIWAN,TAIWAN,1,...,10.1016/0309-586X(77)90003-6,Article,NaN,NaN,Agricultural Administration,1,4,scopus,NaN,NOTREPORTED
4,2010.0,Am. J. Sci.,ACCRETIONARY OROGEN AND EVOLUTION OF THE JAPAN...,Jahn B,The Japanese Islands represent a segment of a ...,"Acad Sinica, Inst Earth Sci, Taipei 11529, Tai...",TAIWAN,TAIWAN,TAIWAN,1,...,10.2475/10.2010.02,Article,A-TYPE GRANITES; ARABIAN-NUBIAN SHIELD; PALEOZ...,AMER JOURNAL SCIENCE,AMERICAN JOURNAL OF SCIENCE,72,310,ISI,INST EARTH SCI,NATL TAIWAN UNIV
5,2021.0,Am. J. Sci.,American Journal of Science,Chang Q;Hren M;Lin A;Tabor C;Yu S;Eley Y;Harris G,Fluvial sediments are important archives of pa...,"Univ Connecticut, Dept Geosci, Storrs, CT 0626...",USA;TAIWAN;TAIWAN;UNITED KINGDOM,USA,USA,7,...,10.2475/04.2021.01,Article,CARBON-ISOTOPE DISCRIMINATION; LAST GLACIAL MA...,AMER JOURNAL SCIENCE,AMERICAN JOURNAL OF SCIENCE,1,321,ISI,UNIV CONNECTICUT;NATL CENT UNIV;EXPLORAT AND D...,UNIV CONNECTICUT
6,2006.0,Anatolia,Assessing a river tracing behavioural model A ...,Lee T,NaN,Graduate School of Leisure and Exercise Studie...,TAIWAN,TAIWAN,TAIWAN,1,...,10.1080/13032917.2006.9687194,Article,model; river; tourism; tourist behavior; t...,Anatolia,Anatolia,9,17,scopus,NATIONAL YUNLIN UNIVERSITY OF SCIENCE AND TECH...,電子郵件: THLEE@YUNTECH.EDU.TW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321,1988.0,NaN,MICROSTRUCTURE OF CHROMIUM SPINEL AND PYRITE A...,HWANG S;SHEN P;CHU H;JENG R,NaN,"CENT GEOL SURVEY,TAIPEI,TAIWAN.;",TAIWAN,TAIWAN,TAIWAN,4,...,10.3406/bulmi.1988.8093,Article,NaN,MASSON EDITEUR,BULLETIN DE MINERALOGIE,4,111,ISI,NaN,NATL SUN YAT SEN UNIV
1324,1996.0,NaN,Comparing the IRI prediction with the observed...,Cheng K;Huang Y,We compare the equatorial anomaly in the Total...,"MOTC, DIRECTORATE GEN TELECOMMUN, TAIPEI 106, ...",TAIWAN,TAIWAN,TAIWAN,2,...,10.1016/0273-1177(95)00933-7,Article; Proceedings Paper,NaN,PERGAMON PRESS LTD,LOW AND EQUATORIAL LATITUDES IN THE INTERNATIO...,5,18,ISI,NaN,TELECOMMUN TRAINING INST
1326,2001.0,NaN,Ionospheric foF2 variations prior to strong ea...,Chuo Y;Chen Y;Liu J;Pulinets S,Many studies of the seismo-ionospheric couplin...,"Natl Cent Univ, Inst Space Sci, Chungli 320, T...",TAIWAN;TAIWAN;TAIWAN;RUSSIA,TAIWAN,TAIWAN,4,...,10.1016/S0273-1177(01)00209-5,Article; Proceedings Paper,CANNOT EARTHQUAKES; LIGHTS; PREDICTION,ELSEVIER SCIENCE BV,ADVANCES IN REMOTE SENSING OF THE MIDDLE AND U...,19,27,ISI,NATL CENT UNIV;NATL CENT UNIV;NATL CENT UNIV;I...,NATL CENT UNIV
1329,2004.0,NaN,Taiwans cooperative space activities at presen...,Ip W,Taiwan is developing a long-term space program...,"Natl Cent Univ, Inst Astron & Space Sci, Chung...",TAIWAN,TAIWAN,TAIWAN,1,...,10.1016/j.asr.2003.04.055,Article; Proceedings Paper,NaN,PERGAMON-ELSEVIER SCIENCE LTD,SPACE SCIENCE EDUCATION AND PROMOTING NORTH-SO...,0,34,ISI,NATL CENT UNIV,NATL CENT UNIV


可以看到有 940 筆條目。

**這是還沒有剔除非目標文章的子集，因此有相當一部份會被歸到 A 類文章，剩下的才能被歸到 B 類文章。**

對於 `AU_CO` 不含 `TAIWAN` 關鍵字的條目，因資料庫紀錄可能有誤，需要手動確認歸屬的類別。

In [8]:
df_good_auco.loc[~contains_taiwan]

,PY,JI,TI,AU,AB,C1,AU_CO,AU1_CO,AUc_CO,N_AU,...,DI,DT,ID,PU,SO,TC,VL,DB,AU_UN,AUc_UN
75,2002.0,Catena,Frequency and spatial distribution of landslid...,Chang J;Slaymaker O,Maps from 1904 and 1915 and air photographs fr...,"Univ British Columbia, Dept Geog, Vancouver, B...",CANADA,CANADA,TAIWAN,2,...,10.1016/S0341-8162(01)00157-6,Article,EVOLUTION,ELSEVIER SCIENCE BV,CATENA,32,46,ISI,UNIV BRITISH COLUMBIA,NATL TAIWAN NORMAL UNIV
130,1994.0,Chem. Geol.,LATE CENOZOIC BASALTIC VOLCANISM AROUND THE TA...,CHUNG S;SUN S;TU K;CHEN C;LEE C,Late Cenozoic intraplate basaltic volcanism in...,"AUSTRALIAN GEOL SURVEY ORG,CANBERRA,ACT 2601,A...",AUSTRALIA;FRANCE,AUSTRALIA,TAIWAN,5,...,10.1016/0009-2541(94)90101-5,Article,EASTERN CHINA; ISOTOPIC EVIDENCE; RIO-GRANDE; ...,ELSEVIER SCIENCE BV,CHEMICAL GEOLOGY,149,112,ISI,UNIV ILLINOIS,NATL TAIWAN UNIV
303,1995.0,Geology,ND ISOTOPIC STUDY OF PRECAMBRIAN BASEMENT OF S...,LAN C;LEE T;ZHOU X;KWON S,We carried out a Sm-Nd isotopic reconnaissance...,"ACAD SINICA, INST GEOL, BEIJING, PEOPLES R CHI...",CHINA;KOREA,CHINA,TAIWAN,4,...,10.1130/0091-7613(1995)023<0249:NISOPB>2.3.CO;2,Article,TECTONIC EVOLUTION; CHINA; ROCKS; GEOCHEMISTRY...,"GEOLOGICAL SOC AMER, INC",GEOLOGY,70,23,ISI,INST GEOL;YONSEI UNIV,INST EARTH SCI
884,1999.0,J. Geosci. Educ.,Earth science student attitudes toward a const...,Chang C;Hua H;Barufaldi J,The notion of constructivism is serving as the...,"Science Education Center, SZB 340, University ...",USA,USA,TAIWAN,3,...,10.5408/1089-9995-47.4.331,Article,Earth science; education; Taiwan,National Association of Geoscience Teachers Inc.,Journal of Geoscience Education,8,47,scopus,AUSTIN,DEPARTMENT OF EARTH SCIENCES
952,1996.0,J. Southeast Asian Earth Sci.,Subductionrelated granitic rocks of Taiwan,Lan C;Jahn B;Mertzman S;Wu T,Late Cretaceous granitic rocks constitute an e...,"UNIV RENNES 1, INST GEOL, F-35042 RENNES, FRAN...",FRANCE;USA;CANADA,FRANCE,TAIWAN,4,...,10.1016/S0743-9547(96)00017-7,Article,TRACE-ELEMENT; PARTITION-COEFFICIENTS; S-TYPE;...,PERGAMON-ELSEVIER SCIENCE LTD,JOURNAL OF SOUTHEAST ASIAN EARTH SCIENCES,65,14,ISI,FRANCE.;FRANKLIN AND MARSHALL COLL;UNIV WESTER...,INST EARTH SCI
1050,1997.0,Lithos,Major and trace element and SrNd isotope const...,Chung S;Cheng H;Jahn B;OReilly S;Zhu B,Paleogene volcanic rocks crop out in three sed...,"CHINESE ACAD SCI,INST GEOL,BEIJING,PEOPLES R C...",CHINA;FRANCE;AUSTRALIA;CHINA,CHINA,TAIWAN,5,...,10.1016/S0024-4937(97)00028-5,Article,CALC-ALKALINE MAGMATISM; EAST TAIWAN OPHIOLITE...,ELSEVIER SCIENCE BV,LITHOS,92,40,ISI,INST GEOL;MACQUARIE UNIV;INST GEOCHEM,NATL TAIWAN UNIV
1212,1990.0,Radio Sci.,A STUDY OF THE CHARACTERISTICS OF VHF RADAR EC...,CHU Y;HSU T;CHEN L;CHAO J;LIU C;ROTTGER J,NaN,"UNIV ILLINOIS,DEPT ELECT & COMP ENGN,URBANA,IL...",SWEDEN,SWEDEN,TAIWAN,6,...,10.1029/RS025i004p00527,Article,NaN,AMER GEOPHYSICAL UNION,RADIO SCIENCE,9,25,ISI,UNIV ILLINOIS;EISCAT SCI ASSOC,NATL CENT UNIV


------------------

# 備用程式碼

In [40]:
for idx, row in df.iterrows():
    doi_str = row['DI']
    api_url = 'https://api.crossref.org/works/' + doi_str
    try: 
        with urlopen(api_url) as url:
            word_json = json.loads(url.read().decode())
        if word_json['message']['title']:
            df.loc[idx, 'Title_verified'] = word_json['message']['title'][0]
        else:
            df.loc[idx, 'Title_verified'] = np.nan
    except HTTPError:
        df.loc[idx, 'Title_verified'] = np.nan

# tmp = df.loc[1, 'DI']
# tmp

/home/jovyan/.conda/envs/jme/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [52]:
# df = df.dropna(subset=['Title_verified'])
# df

In [53]:
df.loc[8]

PY                                                   2008.0
JI                                                 Anatolia
TI         Determining the motivation of wellness travelers
AU                                Chen J;Prebensen N;Huan T
AB        This research depicts a new investigative them...
C1        Department of Recreation Park and Tourism Stud...
AU_CO                               INDIA;USA;NORWAY;TAIWAN
AU1_CO                                                INDIA
AUc_CO                                                  USA
N_AU                                                      3
AU_TW                                                   4.0
FU_TW                                                   NaN
RP        Chen, J.S.; Department of Recreation Park and ...
FU                                                      NaN
FX                                                 NA NA NA
DE         Motivation;  Personal health;  Taiwan;  Wellness
DI                            10.1080/13

In [54]:
# cols = df.columns.tolist()
# newcols = cols[:3] + [cols[-1]] + cols[3:-1]
# df = df[newcols]
# df

In [58]:
df.to_csv('tentative.csv', index=False)

In [55]:
# word_json['message']

In [56]:
# word_json['message']['title'][0]